In [1]:
from lib.datasets import ATPBind3D

# dataset_orig = ATPBind3D()
# dataset_orig.initialize_mask_and_weights()

In [7]:
dataset = ATPBind3D(to_slice=True)
dataset.initialize_mask_and_weights()

Initialize Undersampling: all ones
Initialize Weighting: all ones


ATPBind3D(
  #sample: 631
  #task: 1
)

In [2]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 0

from atpbind_main import main


In [3]:
main(model_key='esm-t33', valid_fold=0)

batch_size: 8
init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [0]
load model esm-t33, kwargs: {'gpu': 0, 'freeze_esm': False, 'freeze_layer_count': 30}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 472, valid samples: 118, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8
0m28s {'sensitivity': 0.6156, 'specificity': 0.9818, 'accuracy': 0.9628, 'precision': 0.6487, 'mcc': 0.6124, 'micro_auroc': 0.946, 'valid_mcc': 0.584, 'train_bce': 0.1278, 'valid_bce': 0.095, 'lr': 0.00084}
0m27s {'sensitivity': 0.6045, 'specificity': 0.9889, 'accuracy': 0.969, 'precision': 0.749, 'mcc': 0.6571, 'micro_auroc': 0.9412, 'valid_mcc': 0.6346, 'train_bce': 0.0634, 'valid_bce': 0.0863, 'lr': 0.00138}
0m30s {'sensitivity': 0.6077, 'specificity': 0.9853, 'accuracy': 0.9657, 'precision': 0.6927, 'mcc': 0.631, 'micro_auroc': 0.9381, 'valid_mcc': 0.6161, 'train_bce': 0.0347, 'valid_bce': 0.0986, 'lr': 0.00192}
0m3

In [121]:
main(model_key='esm-t33', valid_fold=1)

batch_size: 8
init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [0]
load model esm-t33, kwargs: {'gpu': 0, 'freeze_esm': False, 'freeze_layer_count': 30}
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 472, valid samples: 118, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8
0m29s {'sensitivity': 0.6306, 'specificity': 0.9823, 'accuracy': 0.9663, 'precision': 0.6287, 'mcc': 0.612, 'micro_auroc': 0.9471, 'valid_mcc': 0.5269, 'train_bce': 0.1273, 'valid_bce': 0.1016, 'lr': 0.00084}
0m29s {'sensitivity': 0.592, 'specificity': 0.9905, 'accuracy': 0.9724, 'precision': 0.7472, 'mcc': 0.6512, 'micro_auroc': 0.9458, 'valid_mcc': 0.5719, 'train_bce': 0.0597, 'valid_bce': 0.1122, 'lr': 0.00138}
0m32s {'sensitivity': 0.5564, 'specificity': 0.993, 'accuracy': 0.9732, 'precision': 0.7911, 'mcc': 0.6505, 'micro_auroc': 0.942, 'valid_mcc': 0.5699, 'train_bce': 0.0321, 'valid_bce': 0.1126, 'lr': 0.00192}
0m38s {'sensitivity': 0

In [2]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

CYCLE_SIZE = 10
pipeline = Pipeline(
    dataset='atpbind3d',
    model='esm-t33',
    gpus=[1],
    model_kwargs={
        'gpu': 1,
        'freeze_esm': False,
        'freeze_layer_count': 30,  
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    max_length=550,
)

init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [1]
load model esm-t33, kwargs: {'gpu': 1, 'freeze_esm': False, 'freeze_layer_count': 30}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 356, valid samples: 89, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8


In [3]:
pipeline.train_until_fit(patience=10, max_epoch=10)

0m32s {'sensitivity': 0.6051, 'specificity': 0.9792, 'accuracy': 0.9617, 'precision': 0.5875, 'mcc': 0.5761, 'micro_auroc': 0.9343, 'valid_mcc': 0.5519, 'train_bce': 0.1469, 'valid_bce': 0.0985, 'lr': 0.00084}
0m31s {'sensitivity': 0.6006, 'specificity': 0.9894, 'accuracy': 0.9712, 'precision': 0.7353, 'mcc': 0.6499, 'micro_auroc': 0.9446, 'valid_mcc': 0.6357, 'train_bce': 0.0663, 'valid_bce': 0.081, 'lr': 0.00138}
0m30s {'sensitivity': 0.5781, 'specificity': 0.9887, 'accuracy': 0.9695, 'precision': 0.7156, 'mcc': 0.6277, 'micro_auroc': 0.926, 'valid_mcc': 0.6133, 'train_bce': 0.0356, 'valid_bce': 0.1072, 'lr': 0.00192}
0m31s {'sensitivity': 0.5285, 'specificity': 0.9899, 'accuracy': 0.9684, 'precision': 0.7198, 'mcc': 0.6011, 'micro_auroc': 0.9148, 'valid_mcc': 0.6136, 'train_bce': 0.0332, 'valid_bce': 0.1575, 'lr': 0.00246}
0m31s {'sensitivity': 0.6216, 'specificity': 0.9822, 'accuracy': 0.9653, 'precision': 0.6311, 'mcc': 0.6082, 'micro_auroc': 0.9228, 'valid_mcc': 0.6341, 'train_bc

[{'sensitivity': 0.6051,
  'specificity': 0.9792,
  'accuracy': 0.9617,
  'precision': 0.5875,
  'mcc': 0.5761,
  'micro_auroc': 0.9343,
  'valid_mcc': 0.5519,
  'train_bce': 0.1469,
  'valid_bce': 0.0985,
  'lr': 0.00084},
 {'sensitivity': 0.6006,
  'specificity': 0.9894,
  'accuracy': 0.9712,
  'precision': 0.7353,
  'mcc': 0.6499,
  'micro_auroc': 0.9446,
  'valid_mcc': 0.6357,
  'train_bce': 0.0663,
  'valid_bce': 0.081,
  'lr': 0.00138},
 {'sensitivity': 0.5781,
  'specificity': 0.9887,
  'accuracy': 0.9695,
  'precision': 0.7156,
  'mcc': 0.6277,
  'micro_auroc': 0.926,
  'valid_mcc': 0.6133,
  'train_bce': 0.0356,
  'valid_bce': 0.1072,
  'lr': 0.00192},
 {'sensitivity': 0.5285,
  'specificity': 0.9899,
  'accuracy': 0.9684,
  'precision': 0.7198,
  'mcc': 0.6011,
  'micro_auroc': 0.9148,
  'valid_mcc': 0.6136,
  'train_bce': 0.0332,
  'valid_bce': 0.1575,
  'lr': 0.00246},
 {'sensitivity': 0.6216,
  'specificity': 0.9822,
  'accuracy': 0.9653,
  'precision': 0.6311,
  'mcc': 0.

In [5]:
from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0])):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+100] /= 2
    return final_preds

preds = []
targets = []
for item in test_set:
    pred = infer_sliced(pipeline, item['graph'])
    preds.append(pred)

    dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
    target = pipeline.task.target(next(iter(dataloader)))
    targets.append(target)
    
pred = utils.cat(preds)
target = utils.cat(targets)

metric = pipeline.task.evaluate(pred, target)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.6039,
 'specificity': 0.9908,
 'accuracy': 0.9732,
 'precision': 0.7579,
 'mcc': 0.663,
 'micro_auroc': 0.9316}

In [6]:
pipeline = Pipeline(
    dataset='atpbind3d',
    model='esm-t33',
    gpus=[1],
    model_kwargs={
        'gpu': 1,
        'freeze_esm': False,
        'freeze_layer_count': 30,  
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    max_length=550,
    valid_fold_num=1,
)

init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [1]
load model esm-t33, kwargs: {'gpu': 1, 'freeze_esm': False, 'freeze_layer_count': 30}
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 356, valid samples: 89, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8


In [7]:
pipeline.train_until_fit(patience=10, max_epoch=10)

from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0])):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+100] /= 2
    return final_preds

preds = []
targets = []
for item in test_set:
    pred = infer_sliced(pipeline, item['graph'])
    preds.append(pred)

    dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
    target = pipeline.task.target(next(iter(dataloader)))
    targets.append(target)
    
pred = utils.cat(preds)
target = utils.cat(targets)

metric = pipeline.task.evaluate(pred, target)
round_dict(dict_tensor_to_num(metric), 4)

0m31s {'sensitivity': 0.6113, 'specificity': 0.9767, 'accuracy': 0.9601, 'precision': 0.5553, 'mcc': 0.5617, 'micro_auroc': 0.9359, 'valid_mcc': 0.4853, 'train_bce': 0.1482, 'valid_bce': 0.1034, 'lr': 0.00084}
0m31s {'sensitivity': 0.5801, 'specificity': 0.9917, 'accuracy': 0.973, 'precision': 0.7697, 'mcc': 0.6549, 'micro_auroc': 0.9483, 'valid_mcc': 0.564, 'train_bce': 0.0668, 'valid_bce': 0.0914, 'lr': 0.00138}
0m32s {'sensitivity': 0.5861, 'specificity': 0.9893, 'accuracy': 0.971, 'precision': 0.7234, 'mcc': 0.6364, 'micro_auroc': 0.9428, 'valid_mcc': 0.5666, 'train_bce': 0.0367, 'valid_bce': 0.1043, 'lr': 0.00192}
0m33s {'sensitivity': 0.6588, 'specificity': 0.9871, 'accuracy': 0.9722, 'precision': 0.7093, 'mcc': 0.6691, 'micro_auroc': 0.9412, 'valid_mcc': 0.5934, 'train_bce': 0.0266, 'valid_bce': 0.145, 'lr': 0.00246}
0m34s {'sensitivity': 0.549, 'specificity': 0.9907, 'accuracy': 0.9707, 'precision': 0.7385, 'mcc': 0.6222, 'micro_auroc': 0.9146, 'valid_mcc': 0.5462, 'train_bce':

{'sensitivity': 0.6558,
 'specificity': 0.9881,
 'accuracy': 0.973,
 'precision': 0.7234,
 'mcc': 0.6747,
 'micro_auroc': 0.928}

In [1]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 1
CYCLE_SIZE = 10

pipeline = Pipeline(
    dataset='atpbind3d',
    model='lm-gearnet',
    gpus=[GPU],
    model_kwargs={
        'gpu': GPU,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 30,
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    max_length=550,
    valid_fold_num=0,
)

init pipeline, model: lm-gearnet, dataset: atpbind3d, gpus: [1]
load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 30}
freeze_lm: 30
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 356, valid samples: 89, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8


In [2]:
pipeline.train_until_fit(patience=10, max_epoch=10)

from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0])):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+100] /= 2
    return final_preds

preds = []
targets = []
for item in test_set:
    pred = infer_sliced(pipeline, item['graph'])
    preds.append(pred)

    dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
    target = pipeline.task.target(next(iter(dataloader)))
    targets.append(target)
    
pred = utils.cat(preds)
target = utils.cat(targets)

metric = pipeline.task.evaluate(pred, target)
round_dict(dict_tensor_to_num(metric), 4)

1m19s {'sensitivity': 0.5375, 'specificity': 0.9836, 'accuracy': 0.9627, 'precision': 0.6162, 'mcc': 0.5562, 'micro_auroc': 0.872, 'valid_mcc': 0.5164, 'train_bce': 0.159, 'valid_bce': 0.1279, 'lr': 0.00084}
0m57s {'sensitivity': 0.6231, 'specificity': 0.9849, 'accuracy': 0.968, 'precision': 0.6694, 'mcc': 0.6291, 'micro_auroc': 0.9314, 'valid_mcc': 0.6223, 'train_bce': 0.0794, 'valid_bce': 0.0946, 'lr': 0.00138}
0m57s {'sensitivity': 0.5691, 'specificity': 0.9828, 'accuracy': 0.9634, 'precision': 0.6183, 'mcc': 0.5741, 'micro_auroc': 0.9174, 'valid_mcc': 0.6088, 'train_bce': 0.047, 'valid_bce': 0.1147, 'lr': 0.00192}
0m58s {'sensitivity': 0.6276, 'specificity': 0.9864, 'accuracy': 0.9696, 'precision': 0.6932, 'mcc': 0.6438, 'micro_auroc': 0.9379, 'valid_mcc': 0.6223, 'train_bce': 0.0356, 'valid_bce': 0.0974, 'lr': 0.00246}
0m59s {'sensitivity': 0.5871, 'specificity': 0.9841, 'accuracy': 0.9655, 'precision': 0.6442, 'mcc': 0.597, 'micro_auroc': 0.9353, 'valid_mcc': 0.6363, 'train_bce':

{'sensitivity': 0.6202,
 'specificity': 0.9891,
 'accuracy': 0.9723,
 'precision': 0.7295,
 'mcc': 0.6584,
 'micro_auroc': 0.9257}

In [3]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 1
CYCLE_SIZE = 10

pipeline = Pipeline(
    dataset='atpbind3d',
    model='lm-gearnet',
    gpus=[GPU],
    model_kwargs={
        'gpu': GPU,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 30,
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    max_length=550,
    valid_fold_num=1,
)

init pipeline, model: lm-gearnet, dataset: atpbind3d, gpus: [1]
load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 30}
freeze_lm: 30
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 356, valid samples: 89, test samples: 41
Adam parameter: all
use cyclic lr scheduler
pipeline batch_size: 8


In [4]:
pipeline.train_until_fit(patience=10, max_epoch=10)

from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0])):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+100] /= 2
    return final_preds

preds = []
targets = []
for item in test_set:
    pred = infer_sliced(pipeline, item['graph'])
    preds.append(pred)

    dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
    target = pipeline.task.target(next(iter(dataloader)))
    targets.append(target)
    
pred = utils.cat(preds)
target = utils.cat(targets)

metric = pipeline.task.evaluate(pred, target)
round_dict(dict_tensor_to_num(metric), 4)

0m58s {'sensitivity': 0.5193, 'specificity': 0.9863, 'accuracy': 0.9651, 'precision': 0.6434, 'mcc': 0.5602, 'micro_auroc': 0.8991, 'valid_mcc': 0.4685, 'train_bce': 0.1629, 'valid_bce': 0.133, 'lr': 0.00084}
0m58s {'sensitivity': 0.6261, 'specificity': 0.983, 'accuracy': 0.9668, 'precision': 0.6375, 'mcc': 0.6144, 'micro_auroc': 0.939, 'valid_mcc': 0.5493, 'train_bce': 0.0809, 'valid_bce': 0.0997, 'lr': 0.00138}
0m59s {'sensitivity': 0.5534, 'specificity': 0.9922, 'accuracy': 0.9723, 'precision': 0.7723, 'mcc': 0.6403, 'micro_auroc': 0.9295, 'valid_mcc': 0.5737, 'train_bce': 0.0436, 'valid_bce': 0.1147, 'lr': 0.00192}
0m59s {'sensitivity': 0.6187, 'specificity': 0.9898, 'accuracy': 0.9729, 'precision': 0.742, 'mcc': 0.6637, 'micro_auroc': 0.9351, 'valid_mcc': 0.5678, 'train_bce': 0.0294, 'valid_bce': 0.1116, 'lr': 0.00246}
1m2s {'sensitivity': 0.5549, 'specificity': 0.9906, 'accuracy': 0.9708, 'precision': 0.7377, 'mcc': 0.6253, 'micro_auroc': 0.9226, 'valid_mcc': 0.5801, 'train_bce':

{'sensitivity': 0.6424,
 'specificity': 0.989,
 'accuracy': 0.9732,
 'precision': 0.7351,
 'mcc': 0.6734,
 'micro_auroc': 0.9303}